In [16]:
import pandas as pd
import numpy as np

#countvectorizer model
import nltk
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

#tf-idf model
from sklearn.feature_extraction.text import TfidfVectorizer


from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO

from langdetect import detect


In [17]:
df = pd.read_csv("../data/goodreads_final_cleaned.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/goodreads_final_cleaned.csv'

## EDA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9772 entries, 0 to 9771
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          9772 non-null   object 
 1   bookTitle    9772 non-null   object 
 2   bookImage    9772 non-null   object 
 3   bookDesc     9772 non-null   object 
 4   bookRating   9772 non-null   float64
 5   ratingCount  9772 non-null   int64  
 6   reviewCount  9772 non-null   int64  
 7   Genre        9772 non-null   object 
 8   pageCount    9772 non-null   int64  
 9   Author       9772 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 763.6+ KB


In [4]:
df["bookTitle"].nunique()

9272

In [5]:
df["Author"].nunique()

4140

In [6]:
df["Genre"].nunique()

46

In [7]:
df["Genre"].value_counts()

Fiction                          1861
Fantasy                          1857
Young Adult                      1042
Romance                           650
Classics                          559
Historical-Historical Fiction     504
Nonfiction                        425
Science Fiction                   372
Mystery                           320
Horror                            227
Religion                          178
Sequential Art-Graphic Novels     176
Fantasy-Paranormal                118
Paranormal-Vampires               106
Poetry                            101
History                            98
Thriller                           97
Philosophy                         92
Other                              89
Romance-Paranormal Romance         88
Womens Fiction-Chick Lit           69
Childrens                          65
Short Stories                      56
New Adult                          46
Science                            45
Contemporary                       45
Plays       

### dataframe for analysis

In [10]:
#copy book title, book description, genre, and author
dfNew = df[["bookTitle", "bookDesc", "Genre", "Author"]].copy()
dfNew.head()

,bookTitle,bookDesc,Genre,Author
0,The Hunger Games,"Could you survive on your own in the wild, wit...",Young Adult,Suzanne Collins
1,Harry Potter and the Order of the Phoenix,There is a door at the end of a silent corrido...,Fantasy,J.K. Rowling
2,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,Classics,Harper Lee
3,Pride and Prejudice,Alternate cover edition of ISBN 9780679783268S...,Classics,Jane Austen
4,Twilight,About three things I was absolutely positive.F...,Young Adult,Stephenie Meyer


In [11]:
dfNew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9772 entries, 0 to 9771
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   bookTitle  9772 non-null   object
 1   bookDesc   9772 non-null   object
 2   Genre      9772 non-null   object
 3   Author     9772 non-null   object
dtypes: object(4)
memory usage: 305.5+ KB


## Data Processing and Bag of Words

In [12]:
#data pre-processing to remove stop words, punctuation, white space, and convert all words to lower case from book Description
df_new = dfNew.copy()
df_new['Key_words'] = ''
df_new['bookTitle_key'] = ''
r = Rake()

for index, row in df_new.iterrows():
    r.extract_keywords_from_text(row['bookDesc'])
    key_words_dict_scores = r.get_word_degrees()
    row['Key_words'] = list(key_words_dict_scores.keys())

    r.extract_keywords_from_text(row['bookTitle'])
    key_words_dict_scores = r.get_word_degrees()
    row['bookTitle_key'] = list(key_words_dict_scores.keys())
    
    
df_new.head()

,bookTitle,bookDesc,Genre,Author,Key_words,bookTitle_key
0,The Hunger Games,"Could you survive on your own in the wild, wit...",Young Adult,Suzanne Collins,"[could, survive, wild, every, one, make, sure,...","[hunger, games]"
1,Harry Potter and the Order of the Phoenix,There is a door at the end of a silent corrido...,Fantasy,J.K. Rowling,"[door, end, silent, corridor, ’, haunting, har...","[harry, potter, order, phoenix]"
2,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,Classics,Harper Lee,"[unforgettable, novel, childhood, sleepy, sout...","[kill, mockingbird]"
3,Pride and Prejudice,Alternate cover edition of ISBN 9780679783268S...,Classics,Jane Austen,"[alternate, cover, edition, isbn, 978067978326...","[pride, prejudice]"
4,Twilight,About three things I was absolutely positive.F...,Young Adult,Stephenie Meyer,"[three, things, absolutely, positive, first, e...",[twilight]


In [ ]:
#DROP SYMBOLS

In [ ]:
df_new2 = df_new.copy()

df_new2.head()

In [ ]:
#make columns lower case

df_new2['Genre'] = df_new2['Genre'].str.lower()
df_new2['Author'] = df_new2['Author'].str.lower()

#make columns one string
df_new2['Genre'] = df_new2['Genre'].str.replace(" ","")
df_new2['Author'] = df_new2['Author'].str.replace(" ","")

#add brackets for processing in next step
df_new2['Genre'] = df_new2['Genre'].apply(lambda x: x.split(','))
df_new2['Author'] = df_new2['Author'].apply(lambda x: x.split(','))

df_new2.head()

In [ ]:
#create a bag of words columns with processed columns
df_new3 = df_new2.copy()

df_new3['Bag_of_words'] = ''

columns = ['Genre', 'Author', 'bookTitle_key', 'Key_words']

for index, row in df_new3.iterrows():
    words = ''
    for col in columns:
        words += ' '.join(row[col]) + ' '
    row['Bag_of_words'] = words
    
bag_words = df_new3[['bookTitle','Bag_of_words']]
bag_words.head()

## Count Vectorizer

In [ ]:
#CountVectorizer model
count = CountVectorizer()
count_matrix = count.fit_transform(bag_words['Bag_of_words'])

cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)


In [ ]:
test = count_matrix.toarray()
test

In [ ]:
len(test[0])

In [ ]:
sum(test[0])

In [ ]:
indices = pd.Series(bag_words['bookTitle'])

In [ ]:
def recommend(bookTitle, cosine_sim = cosine_sim):
    recommended_book = []
    idx = indices[indices == bookTitle].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:21].index)
    
    for i in top_10_indices:
        recommended_book.append(list(bag_words['bookTitle'])[i])
        
    return recommended_book

In [ ]:
recommend('To Kill a Mockingbird')

In [ ]:
bagwords2 = bag_words.copy()
bagwords2 = bagwords2.drop_duplicates(subset=['bookTitle'])
bagwords2.info()

In [ ]:
bagwords2.loc[bagwords2['bookTitle']== "Apollyon"]

In [ ]:
#CountVectorizer model
count = CountVectorizer()
count_matrix = count.fit_transform(bagwords2['Bag_of_words'])

cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)
indices = pd.Series(bagwords2['bookTitle'])

def recommend(bookTitle, cosine_sim = cosine_sim):
    recommended_book = []
    idx = indices[indices == bookTitle].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_book.append(list(bagwords2['bookTitle'])[i])
        
    return recommended_book


In [ ]:
recommend('Apollyon')

## TF- IDF

In [ ]:
bag_words.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
#https://practicaldatascience.co.uk/data-science/how-to-create-content-recommendations-using-tf-idf
#bag of words is already made, but this will also down-weight common words that appear across documents
tfidf = TfidfVectorizer(stop_words='english')

# numbers to calculate similarities
tfidf_matrix = tfidf.fit_transform(bag_words['Bag_of_words'])

#calculate cosine matrix
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
#Return a dataframe of content recommendations based on TF-IDF cosine similarity.


def get_recommendations(title, cosine_similarities, limit=10):

    
    # Return indices for the target dataframe column and drop any duplicates
    indices = pd.Series(bag_words.index, index=bag_words['bookTitle']).drop_duplicates()

    # Get the index for the target value
    target_index = indices[title]

    # Get the cosine similarity scores for the target value
    cosine_similarity_scores = list(enumerate(cosine_similarities[target_index]))

    # Sort the cosine similarities in order of closest similarity
    cosine_similarity_scores = sorted(cosine_similarity_scores, key=lambda x: x[1], reverse=True)

    # Return tuple of the requested closest scores excluding the target item and index
    cosine_similarity_scores = cosine_similarity_scores[1:limit+1]

    # Extract the tuple values
    index = (x[0] for x in cosine_similarity_scores)
    scores = (x[1] for x in cosine_similarity_scores)    

    # Get the indices for the closest items
    recommendation_indices = [i[0] for i in cosine_similarity_scores]

    # Get the actutal recommendations
    recommendations = bag_words['bookTitle'].iloc[recommendation_indices]

    # Return a dataframe
    df = pd.DataFrame(list(zip(index, recommendations, scores)), 
                      columns=['index','recommendation', 'cosine_similarity_score']) 

    return df

In [ ]:
get_recommendations('The Hunger Games', cosine_similarities)

In [ ]:
# Return indices for the target dataframe column and drop any duplicates
indices = pd.Series(bag_words.index, index=bag_words['bookTitle']).drop_duplicates()

# Get the index for the target value
target_index = indices['The Hunger Games']
target_index

## Sentiment Analysis

In [ ]:
#genre as sentiment
sent_df = df_new2[["bookTitle","Genre", "Key_words"]].copy()
sent_df.head()

In [ ]:
df_new2["Genre"].value_counts()

## Sentiment Analysis w/ Support Vector Machines (SVM)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report
from scikitplot.metrics import plot_confusion_matrix


In [ ]:
#object of WordNetLemmatizer
lm = WordNetLemmatizer()
def text_transformation(df_col):
    corpus = []
    for item in df_col:
        new_item = re.sub('[^a-zA-Z]',' ',str(item))
        new_item = new_item.lower()
        new_item = new_item.split()
        new_item = [lm.lemmatize(word) for word in new_item if word not in set(stopwords.words('english'))]
        corpus.append(' '.join(str(x) for x in new_item))
    return corpus
corpus = text_transformation(sent_df['Key_words'])


## Sentiment Analysis w/ Naive Bayes Algorithm

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

In [ ]:
count_matrix

In [ ]:
#apply TF_IDF Transformation to the bag of words columns (post CountVectorizer)
tfidf = TfidfTransformer() # by default applies "l2" normalization
X_tfidf = tfidf.fit_transform(count_matrix)
X_tfidf = X_tfidf.todense()
X_tfidf

In [ ]:

#split training and testing data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, 
                                                    test_size = 0.25, 
                                                    random_state = 0)